In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
!pip list

Package                       Version
----------------------------- ---------------------
absl-py                       1.0.0
affine                        2.3.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
alembic                       1.7.5
anyio                         3.4.0
apex                          0.1
appdirs                       1.4.4
argon2-cffi                   21.1.0
asgiref                       3.4.1
async-generator               1.10
async-timeout                 4.0.2
attrs                         21.2.0
audioread                     2.1.9
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.10.0
bleach                        4.1.0
blinker                       1.4
blis                          0.7.5
bokeh                         2.4.2
boto3                         1.20.13
botocore                      1.23.13
brotlip

In [2]:
# TODO: automate with nice data configs of selected variables etc.
# TODO: logging functions

from coralshift.dataloading import config, get_data
from coralshift import functions_creche
from coralshift.processing import spatial_data
from coralshift.machine_learning import baselines
from coralshift.plotting import spatial_plots

from __future__ import annotations

import threading
from collections.abc import Iterator
from datetime import datetime

import coiled
import dask
import dask.array as da
import dask.dataframe as dd
import distributed
import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import xgboost
from dask_ml.metrics import mean_squared_error


data_fp = config.data_folder

# Optuna

In [13]:
FILEPATH = "data/temp_all_df_-50-0.csv"

# Number of parallel optuna jobs to run
N_JOBS = 10
# Total number of converging trials to run across the various jobs
N_TRIALS = 50
# Number of folds in each trial. This also determines the train/test split
# (e.g. N_FOLDS=5 -> train=4/5 of the total data, test=1/5)
N_FOLDS = 5

# Dask worker instance type and number (per cluster)
# Total number of EC2 instances spun up = N_JOBS * N_WORKERS
WORKER_INSTANCE_TYPE = "r6i.large"
N_WORKERS = 1

In [4]:
# num_files = 20
# train_nc_fps = list(Path("data/64_split_datasets").glob("temp_rf*.nc"))[:num_files]
# # test_nc_fps = list(Path("data/64_split_datasets").glob("temp_rf*.nc"))[num_files:num_files + num_files]


# train_Xs, train_ys = [], []
# for i, fp in enumerate(train_nc_fps):
#     ds = xa.open_dataset(fp)
#     X, y = ds_to_ml_ready(ds, remove_rows=True)
#     train_Xs.append(X)
#     train_ys.append(y)
    
# train_Xs = pd.concat(train_Xs, axis=0).to_numpy()
# train_ys = pd.concat(train_ys, axis=0).to_numpy()
dd.read_csv(FILEPATH)


,latitude,longitude,so_january_mean,so_february_mean,so_march_mean,so_april_mean,so_may_mean,so_june_mean,so_july_mean,so_august_mean,so_september_mean,so_october_mean,so_november_mean,so_december_mean,so_january_max,so_january_min,so_february_max,so_february_min,so_march_max,so_march_min,so_april_max,so_april_min,so_may_max,so_may_min,so_june_max,so_june_min,so_july_max,so_july_min,so_august_max,so_august_min,so_september_max,so_september_min,so_october_max,so_october_min,so_november_max,so_november_min,so_december_max,so_december_min,so_time_std,so_jan_std,so_jul_std,so_overall_mean,thetao_january_mean,thetao_february_mean,thetao_march_mean,thetao_april_mean,thetao_may_mean,thetao_june_mean,thetao_july_mean,thetao_august_mean,thetao_september_mean,thetao_october_mean,thetao_november_mean,thetao_december_mean,thetao_january_max,thetao_january_min,thetao_february_max,thetao_february_min,thetao_march_max,thetao_march_min,thetao_april_max,thetao_april_min,thetao_may_max,thetao_may_min,thetao_june_max,thetao_june_min,thetao_july_max,thetao_july_min,thetao_august_max,thetao_august_min,thetao_september_max,thetao_september_min,thetao_october_max,thetao_october_min,thetao_november_max,thetao_november_min,thetao_december_max,thetao_december_min,thetao_time_std,thetao_jan_std,thetao_jul_std,thetao_overall_mean,tos_january_mean,tos_february_mean,tos_march_mean,tos_april_mean,tos_may_mean,tos_june_mean,tos_july_mean,tos_august_mean,tos_september_mean,tos_october_mean,tos_november_mean,tos_december_mean,tos_january_max,tos_january_min,tos_february_max,tos_february_min,tos_march_max,tos_march_min,tos_april_max,tos_april_min,tos_may_max,tos_may_min,tos_june_max,tos_june_min,tos_july_max,tos_july_min,tos_august_max,tos_august_min,tos_september_max,tos_september_min,tos_october_max,tos_october_min,tos_november_max,tos_november_min,tos_december_max,tos_december_min,tos_time_std,tos_jan_std,tos_jul_std,tos_overall_mean,uo_january_mean,uo_february_mean,uo_march_mean,uo_april_mean,uo_may_mean,uo_june_mean,uo_july_mean,uo_august_mean,uo_september_mean,uo_october_mean,uo_november_mean,uo_december_mean,uo_january_max,uo_january_min,uo_february_max,uo_february_min,uo_march_max,uo_march_min,uo_april_max,uo_april_min,uo_may_max,uo_may_min,uo_june_max,uo_june_min,uo_july_max,uo_july_min,uo_august_max,uo_august_min,uo_september_max,uo_september_min,uo_october_max,uo_october_min,uo_november_max,uo_november_min,uo_december_max,uo_december_min,uo_time_std,uo_jan_std,uo_jul_std,uo_overall_mean,vo_january_mean,vo_february_mean,vo_march_mean,vo_april_mean,vo_may_mean,vo_june_mean,vo_july_mean,vo_august_mean,vo_september_mean,vo_october_mean,vo_november_mean,vo_december_mean,vo_january_max,vo_january_min,vo_february_max,vo_february_min,vo_march_max,vo_march_min,vo_april_max,vo_april_min,vo_may_max,vo_may_min,vo_june_max,vo_june_min,vo_july_max,vo_july_min,vo_august_max,vo_august_min,vo_september_max,vo_september_min,vo_october_max,vo_october_min,vo_november_max,vo_november_min,vo_december_max,vo_december_min,vo_time_std,vo_jan_std,vo_jul_std,vo_overall_mean,elevation,seafloor_slope,unep_coral_presence
npartitions=40,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float6

In [17]:
clusters: dict[int, tuple[distributed.Client, dd.DataFrame]] = {}


def get_ddf() -> tuple[distributed.Client, dd.DataFrame]:
    thread_id = threading.get_ident()
    try:
        return clusters[thread_id]
    except KeyError:
        pass

    cluster = coiled.Cluster(
        name=f"xgb-test-{thread_id}",
        worker_vm_types=[WORKER_INSTANCE_TYPE],
        scheduler_vm_types=["m6i.large"],
        package_sync=True,  # align remote packages to local ones
        n_workers=N_WORKERS,
        backend_options={
            "region": "eu-west-2",
            "multizone": True,
        },
        scheduler_options={"idle_timeout": "15 minutes"},
    )

    client = distributed.Client(cluster, set_as_default=False)
    print("Started cluster at", client.dashboard_link)

    with client.as_current():
        # Load feature table generated by Feature Engineering.ipynb
        ddf = dd.read_csv(FILEPATH)

        # Reduce dataset size. Uncomment to speed up the exercise.
        # ddf = ddf.partitions[:20]

        # Under the hood, XGBoost converts floats to `float32`.
        # Let's do it only once here.
        float_cols = ddf.select_dtypes(include="float").columns.tolist()
        ddf = ddf.astype({c: np.float32 for c in float_cols})

        # We need the categories to be known
        categorical_vars = ddf.select_dtypes(include="category").columns.tolist()

        # categorize() reads the whole input and then discards it.
        # Let's read from disk only once.
        ddf = ddf.persist()
        # FIXME https://github.com/dask/dask/issues/9901
        ddf = ddf.categorize(columns=categorical_vars, scheduler=client)

        # We will need to access this multiple times. Let's persist it.
        ddf = ddf.persist()

        clusters[thread_id] = client, ddf
        return client, ddf


In [6]:
# Here we subset data for cross-validation
def make_cv_splits(
    ddf: dd.DataFrame, n_folds: int = N_FOLDS
) -> Iterator[tuple[dd.DataFrame, dd.DataFrame]]:
    frac = [1 / n_folds] * n_folds
    splits = ddf.random_split(frac, shuffle=True)
    for i in range(n_folds):
        train = [splits[j] for j in range(n_folds) if j != i]
        test = splits[i]
        yield dd.concat(train), test

In [7]:
def train_model(study_params: dict[str, float]) -> float:
    scores = []
    client, ddf = get_ddf()
    gt = "unep_coral_presence"

    with client.as_current():
        for train, test in make_cv_splits(ddf):
            y_train = train[gt]
            X_train = train.drop(columns=[gt])
            y_test = test[gt]
            X_test = test.drop(columns=[gt])

            d_train = xgboost.dask.DaskDMatrix(
                client, X_train, y_train, enable_categorical=True
            )
            model = xgboost.dask.train(
                client,
                {"tree_method": "hist", **study_params},
                d_train,
                num_boost_round=4,
                evals=[(d_train, "train")],
            )
            predictions = xgboost.dask.predict(None, model, X_test)
            score = mean_squared_error(
                y_test.to_dask_array(),
                predictions.to_dask_array(),
                squared=False,
                compute=False,
            )
            # Compute predictions and mean squared error for this iteration
            # while we start the next one
            scores.append(score.reshape(1).persist())
            del d_train, model, predictions, score

        scores = da.concatenate(scores).compute()
        return scores.mean()

In [8]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 75, 125),
        "learning_rate": trial.suggest_float("learning_rate", 0.5, 0.7),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 6),
        "max_leaves": trial.suggest_int("max_leaves", 0, 2),
        "max_cat_to_onehot": trial.suggest_int("max_cat_to_onehot", 1, 10),
    }
    return train_model(params)

In [16]:
!cuda version

/bin/bash: cuda: command not found


In [14]:
# Create a single study
start = datetime.now()
study = optuna.create_study(study_name="coral_test_0")
study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS)
print(f"Total time:  {datetime.now() - start}")


╭───────────────────────────────────────── Coiled Cluster ─────────────────────────────────────────╮
│                   ]8;id=449929;https://cloud.coiled.io/clusters/317299?account=orlando-code\https://cloud.coiled.io/clusters/317299?account=orlando-code]8;;\                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────── Overview ───────────────────╮╭──────────────── Configuration ─────────────────╮
│                                                ││                                                │
│ Cluster Name: xgb-nyc-taxi-gbh-140131159766784 ││ Region: eu-west-2                              │
│                                                ││                                                │
│ Scheduler Status: error  .                     ││ Scheduler Instance Type: m6i.large             │
│                                                ││                                                │
│ Dashboard Address:   .                         ││ Worker Instance Type(s): r6i.large (1)         │
│                                                ││                                                │
│                                                ││ Workers Requested: 1                           │
│                                                ││                                                │
╰────────────────────────────────────────────────╯╰────────────────────────────────────────────────╯
╭─────────────────────────────────── (2023/11/21 11:44:19 UTC) ────────────────────────────────────╮
│  Cluster status is error (reason: Scheduler Stopped -> Software build failed -> Conda package    │
│  install failed with the following errors:                                                       │
│                                                                                                  │
│  nothing provides __cuda >=11.2,<12 needed by nccl-2.15.5.1-h0800d71_0                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

[W 2023-11-21 11:44:21,594] Trial 2 failed with parameters: {'n_estimators': 98, 'learning_rate': 0.6267538410794163, 'colsample_bytree': 0.6255746481165945, 'colsample_bynode': 0.7094827347817386, 'colsample_bylevel': 0.7085863426683865, 'reg_lambda': 0.843371391197081, 'max_depth': 5, 'max_leaves': 2, 'max_cat_to_onehot': 8} because of the following error: ClusterCreationError("Cluster status is error (reason: Scheduler Stopped -> Software build failed -> Conda package install failed with the following errors:\n\nnothing provides __cuda >=11.2,<12 needed by nccl-2.15.5.1-h0800d71_0\n\nConsider creating a new environment.\nBy specifying your packages at once, you're more likely to get a consistent set of versions.\n\nIf you use conda:\n\n    conda create -n myenv -c conda-forge coiled package1 package2 package3\n\nIf you use pip/venv, create a new environment and then:\n\n    pip install coiled package1 package2 package3\nor\n    pip install -r requirements.txt\n\nIf that does not sol

LiveError: Only one live display may be active at once

In [12]:
# Tear down running clusters
for client, _ in clusters.values():
    client.shutdown()

# Hyperopt

In [ ]:
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score, mean_squared_error

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']),eval_metric="auc",early_stopping_rounds=10)
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print(f"SCORE: {accuracy}")
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()
# takes a while to run
# best_hyperparams = fmin(fn = objective,
#                         space = space,
#                         algo = tpe.suggest,
#                         max_evals = 100,
#                         trials = trials)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)
